# 第二讲 - 矩阵消元及其与矩阵乘法的关系

- 消元法解方程组
- 矩阵简化
- 反向替代
- 矩阵乘法

## 消元法解方程组

$$x+2y+z=2\quad(1)\\3x+8y+z=12\quad(2)\\4y+z=2\quad(3)$$

提取出矩阵：

$$A=\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}$$

下面先做一些初始化工作：

In [1]:
import numpy as np
from sympy import *
init_printing()
x, y, z = symbols('x y z')
lhs = (x + 2*y + z, 3*x + 8*y + z, 4*y + z)
rhs = (2, 12, 2)
A = np.array([[1, 2, 1], [3, 8, 1], [0, 4, 1]])

### 方程组消元方式

首先消去未知数x，我们用等式(2)减去3倍的等式(1)，我们得到等式(4)：

In [2]:
eq4 = Eq(lhs[1]-3*lhs[0], rhs[1]-3*rhs[0])
eq4

由于等式(3)的x系数为0，因此，等式(3)不需要消除x，也可以认为等式(3)减去0倍的等式(1)，得到等式(5)：

In [3]:
eq5 = Eq(lhs[2]-0*lhs[0], rhs[2]-0*rhs[0])
eq5

此时我们的方程组已经经过消去未知数x，变为：

$$x+2y+z=2\quad(1)\\2y-2z=6\quad(4)\\4y+z=2\quad(5)$$

下面我们利用等式(4)消去等式(5)的未知数y，得到等式(6)：

In [4]:
eq6 = Eq(eq5.args[0]-2*eq4.args[0], eq5.args[1]-2*eq4.args[1])
eq6

这是我们得到了消元后的方程组：

$$x+2y+z=2\quad(1)\\2y-2z=6\quad(4)\\5z=-10\quad(6)$$

等式(6)很容易接触未知数z：

In [5]:
solz = solve(eq6, dict=True)
print(solz[0])

{z: -2}


然后依次将z带入等式(4)，求得y，将y和z带入等式(1)，求得x，最终解出全方程组：

In [6]:
soly = solve(eq4.subs(z, -2), y, dict=True)
print(soly[0])

{y: 1}


In [7]:
solx = solve(Eq(x+2*y+z, 2).subs({y:1, z:-2}), x, dict=True)
print(solx[0])

{x: 2}


In [8]:
print({**solx[0], **soly[0], **solz[0]})

{x: 2, y: 1, z: -2}


### 矩阵消元方式

矩阵消元的方式会显得更加自然和简单，从左上角到右下角沿对角线（称为矩阵的主对角线）依次消除对角线以下的数字（将对角线以下数字变成0）。我们将保留在对角线或以上用来进行消元的数字称为Pivot number。

首先将矩阵第二行减去三倍的第一行：

In [9]:
A[1] -= 3 * A[0]
A

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  4,  1]])

当然我们也可以认为下面还进行了将矩阵的第三行减去0倍的第一行：

In [10]:
A[2] -= 0 * A[0]
A

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  4,  1]])

因此，经过第一次消元，矩阵第一列除了pivot number之外，全部变为0，下面我们继续对角线第二个pivot number。

将矩阵第三行减去2倍的第二行：

In [11]:
A[2] -= 2 * A[1]
A

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  0,  5]])

此时，矩阵已经变为一个上三角矩阵，主对角线以下的元素全部为0.

对于等式的右边 RHS，进行同样的操作，我们得到如下的矩阵方程式：

$$\begin{bmatrix}1&2&1\\0&2&-2\\0&0&5\end{bmatrix}\begin{bmatrix}x\\y\\z\end{bmatrix}=\begin{bmatrix}2\\6\\-10\end{bmatrix}$$

In [12]:
b = np.array([2, 12, 2])
b[1] -= 3 * b[0]
b[2] -= 0 * b[0]
b[2] -= 2 * b[1]
b

array([  2,   6, -10])

下面我们完成反向替代的过程，一下步骤等式左边的矩阵和等式右边的向量会一起进行操作：

In [13]:
# 解出z
A[2] = A[2]/5
b[2] /= 5
print(A, b)

[[ 1  2  1]
 [ 0  2 -2]
 [ 0  0  1]] [ 2  6 -2]


In [14]:
# 解出y
A[1] += 2 * A[2]
b[1] += 2 * b[2]
A[0] -= A[2]
b[0] -= b[2]
A[1] = A[1]/2
b[1] /= 2
print(A, b)

[[1 2 0]
 [0 1 0]
 [0 0 1]] [ 4  1 -2]


In [15]:
# 解出x
A[0] -= 2 * A[1]
b[0] -= 2 * b[1]
print(A, b)

[[1 0 0]
 [0 1 0]
 [0 0 1]] [ 2  1 -2]


上面的结果表示成矩阵方程组（或者更进一步列视角）为：

$$\begin{bmatrix}1&0&0\\0&1&0\\0&0&1\end{bmatrix}\begin{bmatrix}x\\y\\z\end{bmatrix}=\begin{bmatrix}2\\2\\-2\end{bmatrix}\quad\to\quad x\begin{bmatrix}1\\0\\0\end{bmatrix}+y\begin{bmatrix}0\\1\\0\end{bmatrix}+z\begin{bmatrix}0\\0\\1\end{bmatrix}=\begin{bmatrix}2\\1\\-2\end{bmatrix}$$

显而易见，等式右边就是方程组的解。

### 行交换

正如将等式(1)(2)(3)进行位置交换不会影响方程求解结果一样，在矩阵表达中，交换矩阵的行（同时也要交换等式右边RHS的向量位置）也不会影响结果。

我们也可以将上面方程组的矩阵表达方式写成：

$$A=\begin{bmatrix}1&2&1\\0&4&1\\3&8&1\end{bmatrix}\begin{bmatrix}x\\y\\z\end{bmatrix}=\begin{bmatrix}2\\2\\12\end{bmatrix}$$

## 矩阵乘法

假设有n行n列的矩阵A，$A=\begin{bmatrix}a_{11}&\cdots&a_{1n}\\ \vdots & \ddots & \vdots \\ a_{n1}&\cdots&a_{nn}\end{bmatrix}$，按照行视角，我们可以表示为$A=\begin{bmatrix}r_1 \\ \vdots \\ r_n \end{bmatrix}$，按照列视角，我们可以表示为$A=\begin{bmatrix}c_1 & \cdots & c_n \end{bmatrix}$。

如果矩阵A右边乘以一个n长度的向量$x=\begin{bmatrix}x_1\\ \vdots \\ x_n \end{bmatrix}$，按照列视角非常容易表达，结果得到一个n长度的向量：

$$ \begin{bmatrix}c_1 & \cdots & c_n \end{bmatrix}\begin{bmatrix}x_1\\ \vdots \\ x_n \end{bmatrix}=x_1c_1+\cdots+x_nc_n$$

如果矩阵A左边乘以一个$1\times n$的矩阵（向量的倒置）$x=\begin{bmatrix}x_1& \cdots & x_n \end{bmatrix}$，按照行视角非常容易表达，结果得到一个$1\times n$的矩阵（向量的倒置）：

$$ \begin{bmatrix}x_1& \cdots & x_n \end{bmatrix}\begin{bmatrix}r_1 \\ \vdots \\ r_n \end{bmatrix}=x_1r_1+\cdots+x_nr_n$$

**因此当进行矩阵乘法时，结果中的列是乘号左边的矩阵的列的线性组合，结果中的行是乘号右边的矩阵的行的线性组合。**

当我们进行矩阵消元时，我们完全可以依靠矩阵乘法来进行，例如对于矩阵$A=\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}$来说，第一步是需要消去第一列出pivot number外的所有数字，将其变为0。我们已经知道这需要将A的第二行减去3倍的第一行，将A的第三行减去0倍的第一行，结果中第一行保持不变。我们可以先写出乘号左边的第一行，让其是的乘号右边的行的线性组合等于第一行本身，于是可知：

$$\begin{bmatrix}1&0&0\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}1&2&1\end{bmatrix}$$

使用numpy进行验证：

In [16]:
A = np.array([[1, 2, 1], [3, 8, 1], [0, 4, 1]])
np.array([1, 0, 0])@A

array([1, 2, 1])

同理可知，结果第二行应该是矩阵第二行减去3倍第一行所得（或者可以认为是$-3\times r_1 + r_2 + 0\times r_3$）：

$$\begin{bmatrix}-3&1&0\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}0&2&-2\end{bmatrix}$$

结果的第三行应该是矩阵第三行减去0被第一行所得（或者可以认为是$0\times r_1 + 0\times r_2 + r_3$）：

$$\begin{bmatrix}0&0&1\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}0&4&1\end{bmatrix}$$

使用numpy进行验证：

In [17]:
np.array([-3, 1, 0])@A

array([ 0,  2, -2])

In [18]:
np.array([0, 0, 1])@A

array([0, 4, 1])

将这三个水平向量组合成一个矩阵，可以在一次乘法中完成第一个pivot number的消元过程：

$$\begin{bmatrix}1&0&0\\-3&1&0\\0&0&1\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}1&2&1\\0&2&-2\\0&4&1\end{bmatrix}$$

在numpy中进行验证：

In [19]:
fac1 = np.eye(3)
fac1[1, 0] = -3
fac1@A

array([[ 1.,  2.,  1.],
       [ 0.,  2., -2.],
       [ 0.,  4.,  1.]])

类推下去，可以在左边再乘以一个矩阵，完成第二个pivot number的消元过程：

$$\begin{bmatrix}1&0&0\\0&1&0\\0&-2&1\end{bmatrix}\begin{bmatrix}1&0&0\\-3&1&0\\0&0&1\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}1&2&1\\0&2&-2\\0&0&5\end{bmatrix}$$

验证：

In [20]:
fac2 = np.eye(3)
fac2[2, 1] = -2
fac2@fac1@A

array([[ 1.,  2.,  1.],
       [ 0.,  2., -2.],
       [ 0.,  0.,  5.]])

如果继续在左边添加矩阵相乘，我们还能完成反向替代的过程：

$$\begin{bmatrix}1&-2&0\\0&1&0\\0&0&1\end{bmatrix}\begin{bmatrix}1&0&0\\0&\frac{1}{2}&0\\0&0&1\end{bmatrix}\begin{bmatrix}1&0&-1\\0&1&2\\0&0&1\end{bmatrix}\begin{bmatrix}1&0&0\\0&1&0\\0&0&\frac{1}{5}\end{bmatrix}\begin{bmatrix}1&0&0\\0&1&0\\0&-2&1\end{bmatrix}\begin{bmatrix}1&0&0\\-3&1&0\\0&0&1\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}1&0&0\\0&1&0\\0&0&1\end{bmatrix}$$

验证（因为涉及数值计算精度问题，所以最终结果不是完全为0或1，使用round函数目的是为了清晰）：

In [21]:
fac3 = np.eye(3)
fac3[2, 2] = 1.0/5
fac4 = np.array([[1, 0 , -1],[0, 1, 2], [0, 0, 1]])
fac5 = np.eye(3)
fac5[1, 1] = 1.0/2
fac6 = np.eye(3)
fac6[0, 1] = -2
(fac6@fac5@fac4@fac3@fac2@fac1@A).round()

array([[ 1.,  0., -0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

从上面我们可以看到，如果需要一个矩阵将A变为上三角矩阵U的话，我们需要的是fac2乘以fac1：

$$\begin{bmatrix}1&0&0\\-3&1&0\\6&-2&1\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}1&2&1\\0&2&-2\\0&0&5\end{bmatrix}$$

In [22]:
fac2@fac1

array([[ 1.,  0.,  0.],
       [-3.,  1.,  0.],
       [ 6., -2.,  1.]])

### 矩阵逆

如果将fac6依次乘到fac1，得到的结果和A相乘就会变成一个单位矩阵I，根据定义$A^{-1}A=I$，我们可知：

$$fac6 \times fac5 \times fac4 \times fac3 \times fac2 \times fac1 = A^{-1}$$

In [23]:
fac6@fac5@fac4@fac3@fac2@fac1

array([[ 0.4,  0.2, -0.6],
       [-0.3,  0.1,  0.2],
       [ 1.2, -0.4,  0.2]])

### 使用矩阵乘法行交换

当乘法左边是单位矩阵I时，矩阵数据不会发生变化，因此如果我们重新排列单位矩阵的行之后，在与矩阵A相乘，就能对A的行进行交换，例如我们需要交换A的第二行和第三行：

$$\begin{bmatrix}1&0&0\\0&0&1\\0&1&0\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}1&2&1\\0&4&1\\3&8&1\end{bmatrix}$$

验证：

In [24]:
np.array([[1, 0, 0],[0, 0, 1],[0, 1, 0]])@A

array([[1, 2, 1],
       [0, 4, 1],
       [3, 8, 1]])

如果我们需要循环的将行下移一个位置的话：

$$\begin{bmatrix}0&0&1\\1&0&0\\0&1&0\end{bmatrix}\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}=\begin{bmatrix}0&4&1\\1&2&1\\3&8&1\end{bmatrix}$$

验证：

In [25]:
np.array([[[0, 0, 1],[1, 0, 0],[0, 1, 0]]])@A

array([[[0, 4, 1],
        [1, 2, 1],
        [3, 8, 1]]])

通过上面的列视角我们可以知道，如果需要交换列的位置的话，只需要在矩阵A的右边乘以单位矩阵的重新排列即可，如：

$$\begin{bmatrix}1&2&1\\3&8&1\\0&4&1\end{bmatrix}\begin{bmatrix}0&0&1\\1&0&0\\0&1&0\end{bmatrix}=\begin{bmatrix}2&1&1\\8&1&3\\4&1&0\end{bmatrix}$$

验证：

In [26]:
A@np.array([[[0, 0, 1],[1, 0, 0],[0, 1, 0]]])

array([[[2, 1, 1],
        [8, 1, 3],
        [4, 1, 0]]])